In [5]:
import yfinance as yf

etf = yf.Ticker("ACWI")
hist = etf.history(period="max", interval="1mo") 
hist.to_csv("ACWI_history.csv")

In [6]:
# Gold futures spot price
gold = yf.Ticker("GC=F")

# Download daily history
hist_gold = gold.history(period="max", interval="1mo")
hist_gold.to_csv("gold_history.csv")

